In [13]:
import pandas as pd
import numpy as np
import time
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# Hàm load dữ liệu theo batch
def load_data_by_batch(csv_path, batch_id):
    df = pd.read_csv(csv_path)
    df["Bộ"] = df["Bộ"].astype(int)  # Chuyển về kiểu số nguyên nếu cần

    batch_df = df[df["Bộ"] == batch_id]

    n_fft = batch_df["n_fft"].values[0]
    hop_length = batch_df["hop_length"].values[0]

    # Load train data
    train_row = batch_df[batch_df["Tập"] == "train"]
    test_row = batch_df[batch_df["Tập"] == "test"]

    train_x_path = train_row["X_file_path"].values[0]
    train_y_path = train_row["y_file_path"].values[0]
    test_x_path = test_row["X_file_path"].values[0]
    test_y_path = test_row["y_file_path"].values[0]

    # Đọc dữ liệu
    X_train = np.load(train_x_path)
    y_train = np.load(train_y_path)
    X_test = np.load(test_x_path)
    y_test = np.load(test_y_path)

    return X_train, y_train, X_test, y_test, n_fft, hop_length

In [14]:
# Hàm chạy SVM với nhiều mức MFCC (13, 20, 40, 80)
def run_svm_experiment(X_train, y_train, X_test, y_test, n_fft, hop_length, mfcc_steps):
    scaler = StandardScaler()

    # Scale toàn bộ 80 MFCC trước
    num_samples_train = X_train.shape[0]
    num_samples_test = X_test.shape[0]

    X_train_scaled = scaler.fit_transform(X_train.reshape(num_samples_train, -1))
    X_test_scaled = scaler.transform(X_test.reshape(num_samples_test, -1))

    svm = SVC()
    svm_params = {
        'C': [0.1, 1, 10, 100],
        'kernel': ['linear', 'rbf', 'poly'],
    }

    results = []
    for i in mfcc_steps:
        # Lấy i hệ số MFCC đầu tiên
        X_train_subset = X_train_scaled[:, :i]
        X_test_subset = X_test_scaled[:, :i]

        # Đo thời gian chạy
        start_time = time.time()

        # GridSearch với SVM
        grid_search = GridSearchCV(svm, svm_params, cv=5, n_jobs=-1, verbose=1)
        grid_search.fit(X_train_subset, y_train)

        best_params = grid_search.best_params_
        y_pred = grid_search.predict(X_test_subset)
        test_accuracy = accuracy_score(y_test, y_pred)

        elapsed_time = time.time() - start_time

        # Lưu kết quả
        results.append({
            'numofMFCC': i,
            'n_fft': n_fft,
            'Hop_length': hop_length,
            'Best_hyperparameters': best_params,
            'test_accuracy': test_accuracy,
            'time': elapsed_time
        })

    return pd.DataFrame(results)

In [15]:
# Chạy thử nghiệm
mfcc_steps = [13, 20, 40, 80]
csv_path = r"C:\Users\manhm\Desktop\BeeSoundClassifier\data\extracted_features\mfcc\mfcc_extraction_log.csv"

for i in range(0, 4):
    batch_id = i
    data = load_data_by_batch(csv_path, batch_id)
    
    if data is None:
        continue  # Nếu batch không có dữ liệu thì bỏ qua

    X_train, y_train, X_test, y_test, n_fft, hop_length = data
    df = run_svm_experiment(X_train, y_train, X_test, y_test, n_fft, hop_length, mfcc_steps)

    print(f"\n📌 Kết quả Batch {batch_id}:")
    print(df)

    df.to_csv(f"svm_results_batch_{batch_id}.csv", index=False)


Fitting 5 folds for each of 12 candidates, totalling 60 fits
Fitting 5 folds for each of 12 candidates, totalling 60 fits
Fitting 5 folds for each of 12 candidates, totalling 60 fits
Fitting 5 folds for each of 12 candidates, totalling 60 fits

📌 Kết quả Batch 0:
   numofMFCC  n_fft  Hop_length          Best_hyperparameters  test_accuracy  \
0         13   1024         256  {'C': 0.1, 'kernel': 'poly'}       0.781159   
1         20   1024         256  {'C': 0.1, 'kernel': 'poly'}       0.797464   
2         40   1024         256   {'C': 0.1, 'kernel': 'rbf'}       0.811232   
3         80   1024         256  {'C': 0.1, 'kernel': 'poly'}       0.818478   

         time  
0   81.616194  
1   80.152880  
2  118.670844  
3  272.065779  
Fitting 5 folds for each of 12 candidates, totalling 60 fits
Fitting 5 folds for each of 12 candidates, totalling 60 fits
Fitting 5 folds for each of 12 candidates, totalling 60 fits
Fitting 5 folds for each of 12 candidates, totalling 60 fits

📌 Kết quả 

c:\Users\manhm\anaconda3\Lib\site-packages\joblib\externals\loky\process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Fitting 5 folds for each of 12 candidates, totalling 60 fits
Fitting 5 folds for each of 12 candidates, totalling 60 fits
Fitting 5 folds for each of 12 candidates, totalling 60 fits

📌 Kết quả Batch 2:
   numofMFCC  n_fft  Hop_length          Best_hyperparameters  test_accuracy  \
0         13   2048         512  {'C': 0.1, 'kernel': 'poly'}       0.784058   
1         20   2048         512  {'C': 0.1, 'kernel': 'poly'}       0.796377   
2         40   2048         512   {'C': 0.1, 'kernel': 'rbf'}       0.806159   
3         80   2048         512  {'C': 0.1, 'kernel': 'poly'}       0.821014   

         time  
0   75.033172  
1   79.635048  
2  119.454386  
3  284.293639  
Fitting 5 folds for each of 12 candidates, totalling 60 fits
Fitting 5 folds for each of 12 candidates, totalling 60 fits
Fitting 5 folds for each of 12 candidates, totalling 60 fits
Fitting 5 folds for each of 12 candidates, totalling 60 fits

📌 Kết quả Batch 3:
   numofMFCC  n_fft  Hop_length          Best_hyperp